## Company Industry, Emission and Climate Action Information

The goal of this jupyter notebook is to create a final dataframe that contains the following information:
1. Company information such as geography, industry (source: Entelligent)
2. Scope 1, Scope 2, Scope 3 Emissions for the period 2012-2020 (source: Entelligent)
3. Company climate action objective (source: Science_based Target Inititative)
4. Country Emission Information: (Source:Our World In Data)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Step 1** Importing all df and renaming columns name

In [3]:
emissions = pd.read_csv('../data/entelligent/ent_emissions.csv')
bakeoff = pd.read_csv('../data/entelligent/ent_gics.csv')
country_isin = pd.read_csv('../data/web/country_isin.csv')
climate_target = pd.read_csv('../data/web/companies_target.csv')
country_emission = pd.read_csv('../data/output/country_emission.csv')

In [4]:
#cleaning Entelligent Dataframe column title
def lowering (value):
    return value.lower()
#scopes dataset
col_emissions = [col for col in emissions.columns]
col_emissions2 = [lowering(col) for col in emissions.columns]
dictionary_emissions = dict(zip(col_emissions,col_emissions2 ))
emissions.rename(columns=dictionary_emissions, inplace=True)
#gics dataset
col_bakeoff = [col for col in bakeoff.columns]
col_bakeoff2 = [lowering(col) for col in bakeoff.columns]
dictionary_bakeoff = dict(zip(col_bakeoff,col_bakeoff2 ))
bakeoff.rename(columns=dictionary_bakeoff, inplace=True)

**Step 2** Filtering main df by benchmark and columns

*General Note:* The bakeoff dataset is the one that provides the list of companies by benchmark. Thus, this df is the main one where I will be adding predictors coming from the other dfs. 

In [5]:
#Selecting  MSCI ACWI Universe and checking its shape
msci = bakeoff[bakeoff['bench_name']=='ACWI-US'].copy()
msci.shape

(73238, 24)

In [6]:
#checking null values
msci.isnull().mean()

t_date                         0.000000
bench_name                     0.000000
fsym_id                        0.000000
ticker_region                  0.033425
isin_region                    0.036183
proper_name                    0.033412
gics_sector_name               0.081501
gics_industry_group_name       0.081501
gics_industry_name             0.081501
gics_sub_industry_name         0.081501
port_weight                    0.000000
returns                        0.032142
region                         0.048458
emtr                           0.106543
emtr_rank                      0.107035
tod                            0.048458
t_risk_carbon_adjusted         0.035064
quarterly_revenue              0.479041
market_value                   0.216704
carbon_footprint               0.216704
total_tonnes_scopes_1_and_2    0.216295
carbon_related_industry        0.243480
carbon_intensity               0.479573
revenue_per_tonne              0.479041
dtype: float64

In [7]:
#selecting the columns I am interested into 
msci = msci[['t_date', 'fsym_id', 'ticker_region', 'isin_region',
             'gics_sector_name', 'gics_sub_industry_name', 'port_weight',
             'returns', 'region',  'market_value']]

In [8]:
#dropping all null values in ticker region
msci = msci[~msci['ticker_region'].isnull()]

**Step 3** Feature Engineering and Cleaning 

In [9]:
#feature engineering market capitalization as follow
#large cap >10 billions $
#medium cap between 2-10 billions#
#small cap less than 2 billions 
def cap_size(value):
    if value == None:
        return 'missing'
    elif value > 10_000_000_000:
        return 'large'
    elif value <=10_000_000_000 and value>2_000_000_000:
        return 'medium'
    elif value <= 2_000_000_000:
        return 'small'     
cap_size(None)
msci['company_cap'] = msci['market_value'].map(cap_size)

In [10]:
#I want to have a column with the name of the ocuntry companies belong to
#I am creating the isin code from the ticker region. Pulling everything tha comes after the dash
#And I am checking with the isin code that comes from the website that I have a perfect match
msci['country_isin'] = msci['ticker_region'].map(lambda x: str(x).split('-')[-1])
country_isin.drop(columns='name', inplace=True) # deleting name as reduntant information
msci = pd.merge(msci, country_isin, left_on='country_isin', right_on='country_isin', how='left')

In [11]:
msci['country_name'].unique()

array(['Thailand', 'United kingdom', 'Korea, republic of', 'Brazil',
       'Netherlands', 'United states', 'Australia', 'China', 'Sweden',
       'Switzerland', 'Germany', 'Italy', 'Indonesia', 'Japan',
       'Hong kong', 'South africa', 'Philippines', 'Canada', 'Poland',
       'Spain', 'Qatar', 'Singapore', 'France', 'Finland', 'Malaysia',
       'Taiwan, province of china', 'Denmark', 'Turkey', 'Mexico',
       'Belgium', 'Norway', 'Russian federation', 'New zealand',
       'Portugal', 'Chile', 'Czech republic', 'Colombia', 'India',
       'Austria', 'Saudi arabia', 'Greece', 'Israel',
       'United arab emirates', 'Egypt', 'Hungary', 'Ireland', 'Pakistan',
       'Kuwait'], dtype=object)

In [12]:
msci['country_isin'].unique()

array(['TH', 'GB', 'KR', 'BR', 'NL', 'US', 'AU', 'CN', 'SE', 'CH', 'DE',
       'IT', 'ID', 'JP', 'HK', 'ZA', 'PH', 'CA', 'PL', 'ES', 'QA', 'SG',
       'FR', 'FI', 'MY', 'TW', 'DK', 'TR', 'MX', 'BE', 'NO', 'RU', 'NZ',
       'PT', 'CL', 'CZ', 'CO', 'IN', 'AT', 'SA', 'GR', 'IL', 'AE', 'EG',
       'HU', 'IE', 'PK', 'KW'], dtype=object)

In [13]:
#Now I also would like to map better the region; to do that I am creating a dictionary and then applying a function
#on the country name I just added 
country_region = { 'Thailand': 'Asia/Pacific Ex Japan', 'United kingdom': 'Europe', 
                  'Korea, republic of': 'Asia/Pacific Ex Japan', 'Brazil': 'Latin America',
                  'Netherlands': 'Europe', 'United states':'North America', 
                  'Australia': 'Asia/Pacific Ex Japan', 'China': 'Asia/Pacific Ex Japan', 
                  'Sweden': 'Europe','Switzerland': 'Europe', 'Germany': 'Europe', 'Italy': 'Europe', 
                  'Indonesia': 'Asia/Pacific Ex Japan', 'Japan':'Japan','Hong kong': 'Asia/Pacific Ex Japan', 
                  'South africa': 'Africa/Mideast', 'Philippines': 'Asia/Pacific Ex Japan', 
                  'Canada':'North America', 'Poland': 'Europe','Spain': 'Europe', 'Qatar':'Africa/Mideast', 
                  'Singapore': 'Asia/Pacific Ex Japan', 'France': 'Europe', 'Finland': 'Europe', 
                  'Malaysia': 'Asia/Pacific Ex Japan','Taiwan, province of china': 'Asia/Pacific Ex Japan', 
                  'Denmark': 'Europe', 'Turkey': 'Africa/Mideast', 
                  'Mexico':'Latin America','Belgium': 'Europe', 'Norway': 'Europe', 
                  'Russian federation': 'Europe', 'New zealand': 'Asia/Pacific Ex Japan','Portugal': 'Europe', 
                  'Chile': 'Latin America', 'Czech republic': 'Europe', 'Colombia': 'Latin America', 
                  'India': 'Asia/Pacific Ex Japan','Austria': 'Europe', 'Saudi arabia': 'Africa/Mideast', 
                  'Greece': 'Europe', 'Israel': 'Africa/Mideast','United arab emirates': 'Africa/Mideast', 
                  'Egypt': 'Africa/Mideast', 'Hungary': 'Europe', 'Ireland': 'Europe', 
                  'Pakistan': 'Asia/Pacific Ex Japan','Kuwait': 'Africa/Mideast'}
def map_region(country):
    if country in country_region.keys():
        return country_region.get(country)
    return np.nan
msci['region'] = msci['country_name'].apply(map_region)

In [14]:
msci.shape

(70790, 13)

In [15]:
msci['region'].unique()

array(['Asia/Pacific Ex Japan', 'Europe', 'Latin America',
       'North America', 'Japan', 'Africa/Mideast'], dtype=object)

In [16]:
#another interesting characteristics that might be worth to explore is whether a company 
#comes from a developed or emerging market; source: https://www.msci.com/our-solutions/indexes/acwi 
#no info on MSCI for russia and pakistan: found here https://www.investopedia.com/terms/e/emergingmarketeconomy.asp
country_economy = { 'Thailand':'emerging', 'United kingdom':'developed', 'Korea, republic of':'emerging',
                  'Brazil':'emerging','Netherlands':'developed', 'United states':'developed', 
                  'Australia':'developed', 'China':'emerging', 'Sweden':'developed','Switzerland':'developed', 
                  'Germany':'developed', 'Italy':'developed', 'Indonesia':'emerging', 'Japan':'developed','Hong kong':'developed', 'South africa':'emerging', 
                  'Philippines':'emerging', 'Canada':'developed', 'Poland':'emerging','Spain':'developed', 'Qatar':'emerging', 'Singapore':'developed', 
                  'France':'developed', 'Finland':'developed', 'Malaysia':'emerging','Taiwan, province of china':'emerging', 
                  'Denmark':'developed', 'Turkey':'emerging', 'Mexico':'emerging','Belgium':'developed', 'Norway':'developed', 
                  'Russian federation':'emerging', 'New zealand':'developed','Portugal':'developed', 
                  'Chile':'emerging', 'Czech republic':'emerging', 'Colombia':'emerging', 
                  'India':'emerging','Austria':'developed', 'Saudi arabia':'emerging', 'Greece':'emerging', 
                  'Israel':'developed','United arab emirates':'emerging', 'Egypt':'emerging', 
                  'Hungary':'emerging', 'Ireland':'developed', 'Pakistan':'emerging','Kuwait':'emerging'}

def map_economy(country):
    if country in country_economy.keys():
        return country_economy.get(country)
    return np.nan
#map_region('Thailand')
msci['country_economy'] = msci['country_name'].apply(map_economy)

In [17]:
#because all other dataframe fundamental and emissions begins in 2012, I will select date from 2012
msci = msci[msci['t_date']>'2012-01-01']

**Step 4** Filtering the company emissions dataset

In [18]:
#selecting the column I am interested into for the emissions df
emissions = emissions[['fsym_id', 't_date','scope_1_tonnes', 'scope_2_tonnes', 'scope_3_tonnes']].copy()

**Step 5** Merging the emissions df with MSCI

In [19]:
#merging the two dataframe as one
emiss_msci = pd.merge(msci,emissions, left_on=['t_date','fsym_id'], right_on = ['t_date','fsym_id'],how='left')

In [20]:
#We have ~ 3060 company
len(emiss_msci['fsym_id'].unique())

3021

In [21]:
#renaming isin_region with company isin for further merging
emiss_msci.rename(columns={'isin_region': 'company_isin'}, inplace=True)

In [22]:
#filling null value with different name to avoid conflict during future merging
emiss_msci['company_isin'].fillna('none', inplace=True)

**Step 6** Cleaning company climate commitments df and merging it with the latest df

In [23]:
#checking the shape
climate_target.shape

(2811, 6)

In [24]:
#checking the df
climate_target.head(1)

,Company Name,ISIN,Near term - Target Status,Near term - Target Classification,Near term - Target Year,Net-Zero Committed
0,OSB Group PLC,GB00BLDRH360,Committed,NaN,NaN,Yes


In [25]:
#checking the null
climate_target.isnull().mean()

Company Name                         0.000000
ISIN                                 0.600142
Near term - Target Status            0.000000
Near term - Target Classification    0.542156
Near term - Target Year              0.551761
Net-Zero Committed                   0.000000
dtype: float64

In [26]:
#renaming columns 
climate_target.rename(columns={'Company Name':'company_name','ISIN':'company_isin', 'Near term - Target Status':'target_status', 
                              'Near term - Target Classification':'target_classification', 
                              'Near term - Target Year':'target_year','Net-Zero Committed':'net_zero_committed'}, inplace=True)
climate_target.drop(columns='company_name', inplace=True)

In [27]:
#checking the different target status
climate_target['target_status'].value_counts(normalize=True)

Committed      0.542512
Targets Set    0.454287
Targets set    0.003202
Name: target_status, dtype: float64

In [28]:
#cleaning the target
def target(value):
    value = value.replace(' ', '_').lower()
    return value
climate_target['target_status'] = climate_target['target_status'].map(target)

In [29]:
#checking the different target status
climate_target['target_status'].value_counts(normalize=True)

committed      0.542512
targets_set    0.457488
Name: target_status, dtype: float64

In [30]:
#filling null value before the merge to avoid conflict
climate_target['company_isin'].fillna('not_available', inplace=True)
climate_target['target_classification'].fillna('not_available', inplace=True)
climate_target['target_year'].fillna('not_available', inplace=True)

In [31]:
#checking null value
climate_target.isnull().mean()

company_isin             0.0
target_status            0.0
target_classification    0.0
target_year              0.0
net_zero_committed       0.0
dtype: float64

In [32]:
# I need to put companies in group according to the way they decided to determine their target;
# companies have listed either one or more than one target year for their emission reducton roadmap
# companies that have listed one single year I group them as either in 2030 or later/earlier tahn 2030
# companies that have listed two or three target years I grouped them as 2030 and earlier/ 2030 and later/ later earlier than 2030
def map_target(value):
    if value == 'not_available':
        return value
    else:
        years = value.split(',')
        years = [int(i) for i in years]

        if len(years)==1:
            if years[0] ==2030:
                return '2030'
            elif years[0] >2030:
                return 'later than 2030'
            elif years[0]<2030:
                return 'earlier than 2030'

        if len(years)>1:
            if max(years) <= 2030:
                return '2030 and earlier'
            elif min(years)>=2030:
                return '2030 and later'
            else:
                return 'earlier and later 2030'

climate_target['target_year'] = climate_target['target_year'].map(map_target)

In [33]:
#adding up status with classification and target year. That's because classification and year are filled in only if status is == target set
#there is only one compnay that committed with classification and year. And I am exluding it form my analysis
climate_target['target_status_class_year'] =  climate_target['target_status'] + '-' + climate_target['target_classification'] +'-' + climate_target['target_year'].copy()
climate_target = climate_target[climate_target['target_status_class_year']!='committed-1.5°C-2030']
climate_target.drop(columns=['target_status', 'target_classification', 'target_year'], inplace=True)

In [34]:
#checking unique value
climate_target['target_status_class_year'].unique()

array(['committed-not_available-not_available', 'targets_set-1.5°C-2030',
       'targets_set-Well-below 2°C-2030',
       'targets_set-1.5°C-not_available',
       'targets_set-Well-below 2°C-not_available',
       'targets_set-1.5°C-2030 and later',
       'targets_set-1.5°C-earlier than 2030',
       'targets_set-2°C-2030 and earlier', 'targets_set-2°C-2030',
       'targets_set-1.5°C-2030 and earlier',
       'targets_set-1.5°C-later than 2030',
       'targets_set-Well-below 2°C-earlier than 2030',
       'targets_set-2°C-earlier than 2030',
       'targets_set-Well-below 2°C-2030 and earlier',
       'targets_set-1.5°C/Well-below 2°C-2030',
       'targets_set-1.5°C-earlier and later 2030',
       'targets_set-Well-below 2°C-later than 2030',
       'targets_set-Well-below 2°C-earlier and later 2030',
       'targets_set-Well-below 2°C-2030 and later',
       'targets_set-2°C-later than 2030'], dtype=object)

In [35]:
climate_target['target_status_class_year'].unique()

array(['committed-not_available-not_available', 'targets_set-1.5°C-2030',
       'targets_set-Well-below 2°C-2030',
       'targets_set-1.5°C-not_available',
       'targets_set-Well-below 2°C-not_available',
       'targets_set-1.5°C-2030 and later',
       'targets_set-1.5°C-earlier than 2030',
       'targets_set-2°C-2030 and earlier', 'targets_set-2°C-2030',
       'targets_set-1.5°C-2030 and earlier',
       'targets_set-1.5°C-later than 2030',
       'targets_set-Well-below 2°C-earlier than 2030',
       'targets_set-2°C-earlier than 2030',
       'targets_set-Well-below 2°C-2030 and earlier',
       'targets_set-1.5°C/Well-below 2°C-2030',
       'targets_set-1.5°C-earlier and later 2030',
       'targets_set-Well-below 2°C-later than 2030',
       'targets_set-Well-below 2°C-earlier and later 2030',
       'targets_set-Well-below 2°C-2030 and later',
       'targets_set-2°C-later than 2030'], dtype=object)

In [36]:
#after a series of ieteration I realized that clustering targets with temperature target does not improve model
#thus I am simply using whetehr company committed or target set
def reducing_info(value):
    value = value.split('-')
    return value[0]
climate_target['target_status_class_year'] = climate_target['target_status_class_year'].map(reducing_info)

In [37]:
emiss_climate_msci = pd.merge(emiss_msci, climate_target, 
                                   left_on =['company_isin'], 
                                   right_on=['company_isin'], 
                                   how='left')

In [38]:
#filling nan for climate information: 
# if a company did not communicate roadmapt to reach netzero commitment filled with not-taking action
# if a company did not commit to netzero filled with not-taking action
emiss_climate_msci['target_status_class_year'].fillna('not_taking_action', inplace=True)
emiss_climate_msci['net_zero_committed'].fillna('not_taking_action', inplace=True)

In [39]:
#checking number of companies that did take action but did not disclose the roadmap
len(emiss_climate_msci[(emiss_climate_msci['target_status_class_year']=='undisclosed')&(emiss_climate_msci['net_zero_committed']!='not_taking_action')])

0

**Step 7** Merging Company-Level Information with Country Emissions

In [40]:
# I need to first change the date as they are slightly different
dates = {'2010-03-31': '2010-03-31', '2010-06-30':'2010-06-30', '2010-09-30':'2010-09-30', '2010-12-31':'2010-12-31',
       '2011-03-31': '2011-03-31', '2011-06-30':'2011-06-30', '2011-09-30':'2011-09-30', '2011-12-30':'2011-12-31',
       '2012-03-30': '2012-03-31', '2012-06-29':'2012-06-30', '2012-09-28':'2012-09-30', '2012-12-31':'2012-12-31',
       '2013-03-28': '2013-03-31', '2013-06-28':'2013-06-30', '2013-09-30':'2013-09-30', '2013-12-31':'2013-12-31',
       '2014-03-31': '2014-03-31', '2014-06-30':'2014-06-30', '2014-09-30':'2014-09-30', '2014-12-31':'2014-12-31',
       '2015-03-31': '2015-03-31', '2015-06-30':'2015-06-30', '2015-09-30':'2015-09-30', '2015-12-31':'2015-12-31',
       '2016-03-31': '2016-03-31', '2016-06-30':'2016-06-30', '2016-09-30':'2016-09-30', '2016-12-30':'2016-12-31',
       '2017-03-31': '2017-03-31', '2017-06-30':'2017-06-30', '2017-09-29':'2017-09-30', '2017-12-29':'2017-12-31',
       '2018-03-29': '2018-03-31', '2018-06-29':'2018-06-30', '2018-09-28':'2018-09-30', '2018-12-31':'2018-12-31',
       '2019-03-29': '2019-03-31', '2019-06-28':'2019-06-30', '2019-09-30':'2019-09-30', '2019-12-31':'2019-12-31',
       '2020-03-31': '2020-03-31', '2020-06-30':'2020-06-30', '2020-09-30':'2020-09-30', '2020-12-31':'2020-12-31',
       '2021-03-31': '2021-03-31', '2021-06-30':'2021-06-30', '2021-09-30':'2021-09-30', '2021-12-31':'2021-12-31',
       '2022-03-31': '2022-03-31'}
def map_date(date):
    if date in dates.keys():
        return dates.get(date)
    return np.nan
emiss_climate_msci['t_date'] = emiss_climate_msci['t_date'].apply(map_date)

In [41]:
#merging
df = pd.merge(emiss_climate_msci, country_emission, 
                                   left_on =['country_isin', 't_date'], 
                                   right_on=['country_isin', 'Year'], 
                                   how='left')

In [42]:
#checking shape 
df.shape

(61214, 40)

In [43]:
#checking head
df.head(1)

,t_date,fsym_id,ticker_region,company_isin,gics_sector_name,gics_sub_industry_name,port_weight,returns,region,market_value,...,industry,manufact_construction,transport,electr_heat,buildings,fugitive_emission,other_fuel_combustion,aviation_shipping,Entity,Year
0,2013-09-30,S7RLQ2-R,CPF.R-TH,TH0101010R14,Consumer Staples,Packaged Foods & Meats,0.02767,-0.065453,Asia/Pacific Ex Japan,NaN,...,9595000.0,12640000.0,15615000.0,28695000.0,1875000.0,3187500.0,5205000.0,3535000.0,Thailand,2013-09-30


**Final Step** I want to feature engineer emissions as follow: i) dividing everything by 1_000 to make them more readable: from tons to thousands of tons; ii) I want to add scope 1 and scope 2 together and separately adding scope 1 2 and 3. In the EDA&ttest jupyter notebook I will be then running 5 different ttest to understand which targets will work better for both regression and classification modeling  

In [44]:
#Deviding everything by 1_000
df['scope_1_tonnes']=df['scope_1_tonnes'].map(lambda x: x/1_000)
df['scope_2_tonnes']=df['scope_2_tonnes'].map(lambda x: x/1_000)
df['scope_3_tonnes']=df['scope_3_tonnes'].map(lambda x: x/1_000) 
#Adding emissions
df['all_emission'] = df['scope_1_tonnes'] + df['scope_2_tonnes'] + df['scope_3_tonnes']
df['scope1_2_emission'] = df['scope_1_tonnes'] + df['scope_2_tonnes']
#removing one row with emission =0
df= df[df['scope1_2_emission']>0]

In [45]:
#filtering further everything before 2021 as company emissions were provided only for period 2012-2020
df = df[df['t_date']<'2021-01-01']
#checking lastly null values and dropping them 
df.isnull().mean()
df.dropna(inplace=True)
#dropping final columns

df.drop(columns=['Year', 'ticker_region', 'company_isin',
                  'market_value', 'country_isin', 'country_name' ], inplace=True)


In [46]:
#Saving Final DF
#df.to_csv('../data/output/msci_company_country.csv', index=False)

In [47]:
df['target_status_class_year'].unique()

array(['not_taking_action', 'targets_set', 'committed'], dtype=object)

In [48]:
df.columns

Index(['t_date', 'fsym_id', 'gics_sector_name', 'gics_sub_industry_name',
       'port_weight', 'returns', 'region', 'company_cap', 'country_economy',
       'scope_1_tonnes', 'scope_2_tonnes', 'scope_3_tonnes',
       'net_zero_committed', 'target_status_class_year', 'emissions_lucf',
       'emissions_factor(kg/kwh)', 'emissions_gdp(kg/$ppp)',
       'emissions_in_trade', 'co2_oil', 'co2_cement', 'co2_coal', 'co2_gas',
       'agriculture', 'land_use_forestry', 'waste', 'industry',
       'manufact_construction', 'transport', 'electr_heat', 'buildings',
       'fugitive_emission', 'other_fuel_combustion', 'aviation_shipping',
       'Entity', 'all_emission', 'scope1_2_emission'],
      dtype='object')

In [50]:
df.dtypes

t_date                       object
fsym_id                      object
gics_sector_name             object
gics_sub_industry_name       object
port_weight                 float64
returns                     float64
region                       object
company_cap                  object
country_economy              object
scope_1_tonnes              float64
scope_2_tonnes              float64
scope_3_tonnes              float64
net_zero_committed           object
target_status_class_year     object
emissions_lucf              float64
emissions_factor(kg/kwh)    float64
emissions_gdp(kg/$ppp)      float64
emissions_in_trade          float64
co2_oil                     float64
co2_cement                  float64
co2_coal                    float64
co2_gas                     float64
agriculture                 float64
land_use_forestry           float64
waste                       float64
industry                    float64
manufact_construction       float64
transport                   

In [51]:
df['returns']

145      0.028823
149      0.054051
157     -0.133634
167     -0.029484
211     -0.209121
           ...   
60972   -0.077764
60973    0.039438
60974   -0.059470
60975    0.097190
60976    0.108994
Name: returns, Length: 46776, dtype: float64